# Text parser for volume two

In [45]:
# DO NOT USE

import os
with open("raw.txt", "r") as infile:
    data = infile.readlines()
    
    # Usable Parts of Section Two
    with open("Southern.txt", 'w') as outfile:
        outfile.writelines(data[3074:18607])
        
    # Usable Parts of Section Three
    with open("Northern.txt", 'w') as outfile:
        outfile.writelines(data[18611:39874])
        
    # Usable Parts of Section Four
    with open("Jiuliyang.txt", 'w') as outfile:
        outfile.writelines(data[39727:43643])

In [32]:
# DO NOT USE

import re


def refine(file):
    # Function to process raw data for future further operation
    
    buffer = None
    
    with open(file, "r+") as f:
        buffer = f.read()
        f.seek(0)
        
        # Delete Watermarks
        buffer = re.sub("Kenneth Dean and Zhenman ZHENG - 978-90-47-44017-8", "", buffer).rstrip()
        buffer = re.sub("Downloaded from Brill.com02/18/2020 04:31:15AM", "", buffer).rstrip()
        buffer = re.sub("via National University of Singapore - YNC Library", "", buffer).rstrip()
        
        # Delete Headers
        buffer = re.sub("the southern irrigated plain 南洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the northern irrigated plain 北洋水利系统", "", buffer).rstrip()
        buffer = re.sub("the jiuliyang irrigated plain 九里洋水利系统", "", buffer).rstrip()
        buffer = re.sub("part (one|two|three|four|five)", "", buffer).rstrip()
        
        # Delete Page Numbers
        buffer = re.sub("\n[0-9]+\n", "", buffer).rstrip()
        buffer = re.sub("\f(.*?)\n", "", buffer).rstrip()
        buffer = re.sub("\n\n(A|V)[0-9]+\n\n\n", "", buffer).rstrip()
        
        # Delete blank lines
        buffer = re.sub("\n+", " ", buffer).rstrip()
        
        # Add line breaks at each new entry
        buffer = re.sub("((A|V)[0-9]{3})", "\n\g<1>", buffer)

        f.truncate()        
        f.write(buffer)

In [46]:
# DO NOT USE

refine("Southern.txt")
refine("Northern.txt")
refine("Jiuliyang.txt")



In [41]:
# Read god names and create JSON of them
import os, json

outBuffer= []
godsList = []

with open("list_of_gods.txt", 'r') as infile:
    data = infile.readlines()
    
    for line in data:
        EngText=" ".join(line.split()[:-1]).split("/")
        
        lineEntry = {
            "Name": EngText[0],
            "Name_zh": line.split()[-1]
        }
        
        if len(EngText) > 1:
            lineEntry["Aliases"] = EngText[1:]
        else:
            lineEntry["Aliases"] = []
        
        outBuffer.append(lineEntry)
        
    infile.close()

with open("./god_names.json", "w") as outfile:
                json.dump(outBuffer, outfile, ensure_ascii=False)
                outfile.close()

In [44]:
# Load village mapping functions
import re, os, json

village_index = []

with open("contents.txt") as infile:
    for line in infile:
        buffer = re.split("(V[0-9]{3})", line)      
        entry = {
            "UID_A": buffer[0][0:4],
            "Name": " ".join(buffer[0].split(" ")[1:-2]),
            "Name_zh": buffer[0].split(" ")[-2]
        }

        runs = int((len(buffer)-1)/2 + 1)
        villageList=[]

        for i in range(1,runs):
            village_entry = {
                "UID_V": buffer[i*2-1],
                "Name": buffer[i*2].split(" ")[1],
                "Name_zh": buffer[i*2].split(" ")[2]
            }
            villageList.append(village_entry)

        entry["Num_Villages"] = len(villageList)
        entry["Villages"] = villageList

        village_index.append(entry)

with open("./village_index.json", "w") as outfile:
    json.dump(village_index, outfile, ensure_ascii=False)
    outfile.close()

infile.close()
    
#search function
def whatAlliance(searchterm):
    for alliance in village_index:
        for village in alliance["Villages"]:
            if village["UID_V"] == searchterm:
                return alliance
        
    return -1
    

In [48]:
# Load surname mapping functions

with open('Surname_Groups.json', "r") as surnameFile:
    surnameGroups = json.load(surnameFile)
surnameFile.close()

#search function
def whatSurnames(searchterm):
    for village in surnameGroups:
        if village["UID"] == searchterm:
            return village
    return -1


In [65]:
# Load Text and print them into seperate text files.
import json

villages = []


from pathlib import Path
Path("./Villages").mkdir(parents=True, exist_ok=True)

def createVillageFiles(path, irrNetwork):
    
    with open(path) as infile:
        for line in infile:
            if line[0] == "V":
                uid = line[0:4]                
                buffer1 = line.split(" ", 3)
                buffer2 = buffer1[3].partition("2.")
                buffer3 = buffer2[2].partition("3.")
                buffer4 = buffer3[2].partition("4.")
                buffer5 = buffer4[2].partition("Birthday celebrations of the gods")
            
                # Search for Alliance details
                allianceDetails = whatAlliance(uid)
            
                entry = {
                    "UID": buffer1[0],
                    "Name": buffer1[1],
                    "Name_zh": buffer1[2],
                    "Alliance": allianceDetails["Name"],
                    "Alliance_UID": allianceDetails["UID_A"],
                    "Alliance_zh": allianceDetails["Name_zh"],
                    "Irrigation_Region": irrNetwork,
                    "Village_Settlement": buffer2[0][22:-1],
                    "Surname_Groups": buffer3[0][16:-1],
                    "Village_Temples": buffer4[0][17:-1],
                    "Rituals": {
                        "Yuanxiao processions": buffer5[0][31:-1],
                        "Birthday celebrations of the gods": buffer5[2][1:-2]
                    }
                }
                
                # Add administrative village if applicable
                admin_village = re.search("\w* Administrative Village\s([^\.\s]+)", entry["Village_Settlement"])
                if admin_village != None:
                    split_words = admin_village.group(0).split(" ")
                    entry["Administrative_Village"] = " ".join(split_words[:-1])
                    entry["Administrative_Village_zh"] = "".join(split_words[-1])                    
                else:
                    entry["Administrative_Village"] = "-1"
                    entry["Administrative_Village_zh"] = "-1"
                    
                
                # Find Population
                pop = re.search("[0-9]*(?=\speople)", entry["Village_Settlement"])
                
                if pop != None:
                    entry["population"] = pop.group(0)
                else:
                    entry["population"] = -1
                
                
                # Find surname groups
                if whatSurnames(uid) != -1:
                    entry["Main_Surname_Groups"] = whatSurnames(uid)["Main_Surname_Groups"]
                    entry["Minor_Surname_Groups"] = whatSurnames(uid)["Minor_Surname_Groups"]
                else:
                    entry["Main_Surname_Groups"] = "-1"
                    entry["Minor_Surname_Groups"] = "-1"
                
                villages.append(entry)
                
                
                # Open and edit a new text file for each line
                with open("./Villages/"+uid+".json", "w") as outfile:
                    json.dump(entry, outfile, ensure_ascii=False)
                    outfile.close()
                    
                with open("./village_master.json", "w") as outfile:
                    json.dump(villages, outfile, ensure_ascii=False)
                    outfile.close()
                
        infile.close()
        
        print("completed creating village_master.json")
              

In [64]:
createVillageFiles("./VILLAGE_RAW/1-100.txt", "Southern")


completed creating ./VILLAGE_RAW/1-100.txt
